In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_loss2_with_gammar"
#model = VGG16_quant()
#model_name = "resnet20_quant"
model = VGG16()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    #adjust_list = [150, 225]
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride

In [ ]:
# Using Loss 2 with gammar

In [2]:
gammar = 0.05

In [3]:
def train_loss2(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss1 = criterion(output, target)
        loss2 = model.features[0].weight.abs().sum()
        loss = loss1 + gammar * loss2
        
        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

In [4]:
# This cell won't be given, but students will complete the training

lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train_loss2(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][0/391]	Time 0.693 (0.693)	Data 0.543 (0.543)	Loss 11.0525 (11.0525)	Prec 3.906% (3.906%)
Epoch: [0][100/391]	Time 0.095 (0.100)	Data 0.001 (0.007)	Loss 7.1221 (8.7832)	Prec 11.719% (11.680%)
Epoch: [0][200/391]	Time 0.095 (0.097)	Data 0.002 (0.004)	Loss 4.2916 (6.9747)	Prec 24.219% (14.412%)
Epoch: [0][300/391]	Time 0.095 (0.097)	Data 0.001 (0.004)	Loss 3.3388 (5.9401)	Prec 25.781% (15.885%)
Validation starts
Test: [0/79]	Time 0.449 (0.449)	Loss 2.4740 (2.4740)	Prec 13.281% (13.281%)
 * Prec 12.540% 
best acc: 12.540000
Epoch: [1][0/391]	Time 0.595 (0.595)	Data 0.552 (0.552)	Loss 4.3482 (4.3482)	Prec 19.531% (19.531%)
Epoch: [1][100/391]	Time 0.094 (0.099)	Data 0.002 (0.007)	Loss 3.2926 (3.8897)	Prec 22.656% (20.343%)
Epoch: [1][200/391]	Time 0.094 (0.097)	Data 0.002 (0.004)	Loss 2.9989 (3.7132)	Prec 21.094% (21.311%)
Epoch: [1][300/391]	Time 0.094 (0.096)	Data 0.002 (0.004)	Loss 3.5383 (3.6260)	Prec 25.000% (21.810%)
Validation starts
Test: [0/79]	Time 0.417 (0.417)	Loss 2.

Epoch: [15][200/391]	Time 0.094 (0.097)	Data 0.001 (0.004)	Loss 1.2951 (1.1507)	Prec 78.125% (82.941%)
Epoch: [15][300/391]	Time 0.095 (0.096)	Data 0.002 (0.003)	Loss 1.1410 (1.1483)	Prec 84.375% (82.922%)
Validation starts
Test: [0/79]	Time 0.373 (0.373)	Loss 0.7043 (0.7043)	Prec 74.219% (74.219%)
 * Prec 69.870% 
best acc: 76.820000
Epoch: [16][0/391]	Time 0.650 (0.650)	Data 0.607 (0.607)	Loss 1.2433 (1.2433)	Prec 77.344% (77.344%)
Epoch: [16][100/391]	Time 0.094 (0.100)	Data 0.002 (0.008)	Loss 1.0169 (1.0874)	Prec 83.594% (83.973%)
Epoch: [16][200/391]	Time 0.095 (0.097)	Data 0.001 (0.005)	Loss 1.0815 (1.4197)	Prec 83.594% (83.151%)
Epoch: [16][300/391]	Time 0.095 (0.096)	Data 0.002 (0.004)	Loss 2.6762 (1.3582)	Prec 80.469% (83.282%)
Validation starts
Test: [0/79]	Time 0.313 (0.313)	Loss 1.2721 (1.2721)	Prec 59.375% (59.375%)
 * Prec 56.250% 
best acc: 76.820000
Epoch: [17][0/391]	Time 0.924 (0.924)	Data 0.884 (0.884)	Loss 0.9231 (0.9231)	Prec 89.062% (89.062%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.179 (0.097)	Data 0.003 (0.004)	Loss 0.7298 (0.8354)	Prec 95.312% (90.988%)
Validation starts
Test: [0/79]	Time 0.504 (0.504)	Loss 1.5634 (1.5634)	Prec 54.688% (54.688%)
 * Prec 55.510% 
best acc: 81.420000
Epoch: [31][0/391]	Time 0.838 (0.838)	Data 0.796 (0.796)	Loss 0.8552 (0.8552)	Prec 86.719% (86.719%)
Epoch: [31][100/391]	Time 0.095 (0.102)	Data 0.002 (0.010)	Loss 0.8128 (0.7922)	Prec 91.406% (91.515%)
Epoch: [31][200/391]	Time 0.097 (0.098)	Data 0.002 (0.006)	Loss 0.7222 (0.8034)	Prec 92.188% (91.659%)
Epoch: [31][300/391]	Time 0.094 (0.097)	Data 0.001 (0.004)	Loss 1.1375 (0.8019)	Prec 91.406% (91.629%)
Validation starts
Test: [0/79]	Time 0.344 (0.344)	Loss 0.2977 (0.2977)	Prec 90.625% (90.625%)
 * Prec 85.610% 
best acc: 85.610000
Epoch: [32][0/391]	Time 0.562 (0.562)	Data 0.519 (0.519)	Loss 0.7619 (0.7619)	Prec 91.406% (91.406%)
Epoch: [32][100/391]	Time 0.095 (0.099)	Data 0.002 (0.007)	Loss 0.6973 (0.7988)	Prec 94.531% (91.754%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.441 (0.441)	Loss 0.3744 (0.3744)	Prec 86.719% (86.719%)
 * Prec 83.560% 
best acc: 85.610000
Epoch: [46][0/391]	Time 0.626 (0.626)	Data 0.598 (0.598)	Loss 0.6424 (0.6424)	Prec 94.531% (94.531%)
Epoch: [46][100/391]	Time 0.045 (0.050)	Data 0.001 (0.008)	Loss 0.6385 (0.6744)	Prec 96.094% (95.003%)
Epoch: [46][200/391]	Time 0.043 (0.048)	Data 0.002 (0.005)	Loss 0.6015 (0.6911)	Prec 95.312% (94.648%)
Epoch: [46][300/391]	Time 0.045 (0.047)	Data 0.002 (0.004)	Loss 0.6050 (0.7182)	Prec 98.438% (94.243%)
Validation starts
Test: [0/79]	Time 0.327 (0.327)	Loss 0.5044 (0.5044)	Prec 82.031% (82.031%)
 * Prec 79.870% 
best acc: 85.610000
Epoch: [47][0/391]	Time 0.408 (0.408)	Data 0.386 (0.386)	Loss 0.6509 (0.6509)	Prec 92.969% (92.969%)
Epoch: [47][100/391]	Time 0.044 (0.048)	Data 0.002 (0.006)	Loss 0.8112 (0.7405)	Prec 93.750% (94.299%)
Epoch: [47][200/391]	Time 0.044 (0.046)	Data 0.002 (0.004)	Loss 0.7551 (0.7212)	Prec 91.406% (94.181%)
Epoch: [47][300/391]	

 * Prec 86.300% 
best acc: 86.790000
Epoch: [61][0/391]	Time 0.469 (0.469)	Data 0.447 (0.447)	Loss 0.6309 (0.6309)	Prec 96.875% (96.875%)
Epoch: [61][100/391]	Time 0.044 (0.049)	Data 0.002 (0.006)	Loss 0.6638 (0.6268)	Prec 94.531% (95.993%)
Epoch: [61][200/391]	Time 0.044 (0.047)	Data 0.002 (0.004)	Loss 1.2987 (0.6724)	Prec 97.656% (95.787%)
Epoch: [61][300/391]	Time 0.045 (0.046)	Data 0.001 (0.003)	Loss 0.5701 (0.7100)	Prec 95.312% (95.694%)
Validation starts
Test: [0/79]	Time 0.445 (0.445)	Loss 0.3692 (0.3692)	Prec 87.500% (87.500%)
 * Prec 86.370% 
best acc: 86.790000
Epoch: [62][0/391]	Time 0.451 (0.451)	Data 0.425 (0.425)	Loss 0.5555 (0.5555)	Prec 98.438% (98.438%)
Epoch: [62][100/391]	Time 0.045 (0.049)	Data 0.002 (0.006)	Loss 0.6546 (0.6181)	Prec 95.312% (95.854%)
Epoch: [62][200/391]	Time 0.045 (0.047)	Data 0.002 (0.004)	Loss 0.6577 (0.6320)	Prec 95.312% (95.709%)
Epoch: [62][300/391]	Time 0.045 (0.046)	Data 0.002 (0.003)	Loss 0.7157 (0.6533)	Prec 92.188% (95.590%)
Validation s

Epoch: [76][100/391]	Time 0.045 (0.052)	Data 0.002 (0.009)	Loss 0.7374 (0.7126)	Prec 92.969% (96.117%)
Epoch: [76][200/391]	Time 0.045 (0.048)	Data 0.002 (0.005)	Loss 0.6524 (0.6812)	Prec 94.531% (96.098%)
Epoch: [76][300/391]	Time 0.044 (0.047)	Data 0.002 (0.004)	Loss 0.6990 (0.6625)	Prec 92.969% (96.135%)
Validation starts
Test: [0/79]	Time 0.378 (0.378)	Loss 0.5926 (0.5926)	Prec 82.812% (82.812%)
 * Prec 81.680% 
best acc: 86.790000
Epoch: [77][0/391]	Time 0.419 (0.419)	Data 0.390 (0.390)	Loss 1.0634 (1.0634)	Prec 92.969% (92.969%)
Epoch: [77][100/391]	Time 0.045 (0.048)	Data 0.002 (0.005)	Loss 0.5921 (0.7025)	Prec 95.312% (96.295%)
Epoch: [77][200/391]	Time 0.045 (0.046)	Data 0.002 (0.004)	Loss 0.5894 (0.6357)	Prec 97.656% (96.323%)
Epoch: [77][300/391]	Time 0.045 (0.046)	Data 0.001 (0.003)	Loss 0.6405 (0.6451)	Prec 96.094% (96.260%)
Validation starts
Test: [0/79]	Time 0.241 (0.241)	Loss 0.5503 (0.5503)	Prec 85.938% (85.938%)
 * Prec 83.730% 
best acc: 86.790000
Epoch: [78][0/391]	

Epoch: [91][200/391]	Time 0.044 (0.047)	Data 0.002 (0.004)	Loss 0.1216 (0.1316)	Prec 99.219% (99.510%)
Epoch: [91][300/391]	Time 0.045 (0.046)	Data 0.002 (0.003)	Loss 0.1368 (0.1652)	Prec 100.000% (99.496%)
Validation starts
Test: [0/79]	Time 0.286 (0.286)	Loss 0.3831 (0.3831)	Prec 92.188% (92.188%)
 * Prec 86.860% 
best acc: 90.120000
Epoch: [92][0/391]	Time 0.540 (0.540)	Data 0.488 (0.488)	Loss 0.1459 (0.1459)	Prec 99.219% (99.219%)
Epoch: [92][100/391]	Time 0.045 (0.050)	Data 0.001 (0.006)	Loss 0.1469 (0.1336)	Prec 100.000% (99.482%)
Epoch: [92][200/391]	Time 0.045 (0.047)	Data 0.001 (0.004)	Loss 0.1108 (0.1396)	Prec 99.219% (99.499%)
Epoch: [92][300/391]	Time 0.045 (0.046)	Data 0.002 (0.003)	Loss 0.1070 (0.1345)	Prec 100.000% (99.517%)
Validation starts
Test: [0/79]	Time 0.361 (0.361)	Loss 0.5391 (0.5391)	Prec 86.719% (86.719%)
 * Prec 82.470% 
best acc: 90.120000
Epoch: [93][0/391]	Time 0.511 (0.511)	Data 0.451 (0.451)	Loss 0.1423 (0.1423)	Prec 99.219% (99.219%)
Epoch: [93][100/39

In [5]:
#PATH = "result/VGG16_quant/model_best.pth.tar"
PATH = "result/VGG16_loss2_with_gammar/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9012/10000 (90%)



In [7]:
print(model.features[0].weight.abs().sum())

tensor(1.7484, device='cuda:0', grad_fn=<SumBackward0>)
